In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import tensorflow as tf
from typing import Tuple

Now that we understand how to get the data and model, we're ready to train the neural network. First, we need to load the data using the technique we discussed in the first unit. In order not to clutter this notebook, we've added the `get_data` function and `NeuralNetwork` class you've already seen to a separate `kintro.py` file, which we'll import here. 

In [2]:
!wget -Nq https://raw.githubusercontent.com/MicrosoftDocs/tensorflow-learning-path/main/intro-keras/kintro.py
from kintro import *

In order to understand what happens during training, we need to add a little more detail to our neural network visualization:

为了了解训练过程中发生了什么，我们需要在神经网络可视化中添加更多细节：

![Basic neural network with details](./images/2-fashion-nn-with-details.png)

There's a lot of new information in this diagram, so I'll expand on each of the new concepts here. 

Notice that we've added weights $W$ to the connections between layers, and bias $b$ as input to `Dense` layers &mdash; $W$ and $b$ are the neural network's parameters. Our goal when training our network (also known as fitting) is to find the parameters $W$ and $b$ that minimize the differences between the actual and predicted labels for our data. 

Notice also that we added a Loss function to the diagram. This function takes in the outputs of the model $y'$ (the predicted labels) and the actual labels $y$, measures their differences, and combines those into a single output, which we call the loss. The loss gives us a single number that quantifies how similar our predictions are to the actual labels: a high loss indicates that they're different, and a low loss indicates that our predictions are accurate. The loss function most frequently used in classification is cross-entropy loss, and Keras provides several varieties of this loss. Because our dataset includes a single integer label for each example, we use the [`SparseCategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy) loss function.

Now let's look at what happens in the `Dense` layers. `Dense` layers add a linear operation involving the input data and parameters $W$ and $b$. For example, the top node of the first `Dense` layer performs the following calculation:

此图中有很多新信息，因此我将在此处详细介绍每个新概念。

请注意，我们已将权重 $W$ 添加到层之间的连接，并将偏置 $b$ 作为`Dense`层的输入 &mdash; $W$ 和 $b$ 是神经网络的参数。 训练我们的网络（也称为拟合）时，我们的目标是找到参数 $W$ 和 $b$，以最小化我们数据的实际标签和预测标签之间的差异。

另请注意，我们在图中添加了一个损失函数。 此函数接收模型 $y'$（预测标签）和实际标签 $y$ 的输出，测量它们的差异，并将它们组合成一个输出，我们称之为损失。 损失给了我们一个单一的数字来量化我们的预测与实际标签的相似程度：高损失表示它们不同，低损失表示我们的预测是准确的。 分类中最常用的损失函数是交叉熵损失，Keras 提供了几种这种损失。 因为我们的数据集包含每个示例的单个整数标签，所以我们使用 [`SparseCategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy) 损失函数。

现在让我们看看在`Dense`层中发生了什么。 `Dense` 层添加涉及输入数据和参数 $W$ 和 $b$ 的线性运算。 例如，第一个 `Dense` 层的顶部节点执行以下计算：

$$
z^1_1 = w^1_{1,1} x_1 + ... + w^1_{1,784} x_{784} + b^1_1
$$

If we specify a ReLU activation function, the output of the linear operation is then passed as input to a ReLU function:

如果我们指定 ReLU 激活函数，则线性运算的输出将作为输入传递给 ReLU 函数：

$$
a^1_1 = ReLU(z^1_1)
$$

Mathematically speaking, we can now think of our neural network as a function $\ell$ that takes as input the data $X$, expected labels $y$, and parameters $W$ and $b$, then performs a sequence of operations on that data, and returns a loss. 

从数学上讲，我们现在可以将我们的神经网络视为一个函数 $\ell$，它将数据 $X$、预期标签 $y$ 以及参数 $W$ 和 $b$ 作为输入，然后执行一系列操作 在该数据上，并返回损失。

$$
\mathrm{loss} = \ell(X, y, W, b)
$$

Our goal is to find the parameters $W$ and $b$ that lead to the lowest possible loss. (We can't change our data $X$ or the corresponding labels $y$ &mdash; they're fixed &mdash; but we can adjust $W$ and $b$.) It turns out that problems of this kind fall in the well-studied mathematical area of optimization. The simplest minimization algorithm is gradient descent, and in this sample we use a variation known as Stochastic Gradient Descent or [`SGD`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD). The training process is roughly as follows: First, the parameters $W$ and $b$ are initialized to random values. We then make predictions using those values, and evaluate the loss function to measure the quality of the predictions. And last, we use the optimizer to adjust the values of $W$ and $b$ such that the loss will be a bit lower next time. We repeat this process until we've found parameters that give us a very small loss value and good predictions.

We're now ready to "compile" the model &mdash; this is where we tell it that we want to use the `SGD` optimizer and the `SparseCategoricalCrossentropy` loss function. We also tell the model that we want it to report on the accuracy during training. 

我们的目标是找到导致尽可能低损失的参数 $W$ 和 $b$。 （我们无法更改我们的数据 $X$ 或相应的标签 $y$ &mdash; 它们是固定的 &mdash; 但我们可以调整 $W$ 和 $b$。）事实证明，这类问题属于 深入研究的优化数学领域。 最简单的最小化算法是梯度下降，在此示例中，我们使用称为随机梯度下降或 [`SGD`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD) 的变体 . 训练过程大致如下：首先将参数$W$和$b$初始化为随机值。 然后我们使用这些值进行预测，并评估损失函数以衡量预测的质量。 最后，我们使用优化器调整 $W$ 和 $b$ 的值，以便下次损失会低一些。 我们重复这个过程，直到我们找到可以为我们提供非常小的损失值和良好预测的参数。

我们现在准备好“编译”模型了—— 这是我们告诉它我们要使用 SGD 优化器和 `SparseCategoricalCrossentropy` 损失函数的地方。 我们还告诉模型我们希望它报告训练期间的准确性。

In [3]:
learning_rate = 0.1
batch_size = 64

(train_dataset, test_dataset) = get_data(batch_size)

model = NeuralNetwork()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['accuracy']
model.compile(optimizer, loss_fn, metrics)

A few details from the code above deserve a quick explanation. 

Notice that we pass `from_logits=True` to the loss function. This is because the categorical cross-entropy function requires a probability distribution as input, meaning that the numbers should be between zero and one, and they should add up to one. Our network produces a vector of numbers that have no upper or lower bound (called "logits"), so we need to normalize them to get a probability distribution. This is typically done using the `softmax` function, and specifying `from_logits=True` automatically calculates the softmax before computing the loss.

Notice also that we pass a `learning_rate` to the `SGD` optimizer. The learning rate is a parameter needed in the gradient descent algorithm. We could have left it at the default, which is 0.01, but it's important to know how to specify it because different learning rates can lead to very different prediction accuracies.

Finally, notice that we specified a `batch_size`, which we used in the construction of the `Dataset`, as we saw earlier. This is important during training, because it tells the model that we want to train on 64 images at a time. You might be wondering why 64? Why not train on a single image at a time? Or all 60,000 images at once? Doing a complete training step for each individual image would be inefficient because we would have to perform all the calculations 60,000 times in order to account for every input image. If we included all the input images in $X$, we'd need a lot of memory, and we'd spend a lot of time computing each training step. So we settle for a size in between, called the "mini-batch" size. 

Now that we've configured our model with the parameters we need for training, we can call `fit` to train the model. We specify the number of epochs as 5, which means that we want to iterate over the complete set of 60,000 training images five times while training the neural network. 


上面代码中的一些细节值得快速解释一下。

请注意，我们将`from_logits=True`传递给了损失函数。 这是因为分类交叉熵函数需要一个概率分布作为输入，这意味着数字应该介于 0 和 1 之间，并且它们加起来应该是 1。 我们的网络生成一个没有上限或下限的数字向量（称为`logits`），因此我们需要对它们进行归一化以获得概率分布。 这通常是使用`softmax`函数完成的，指定`from_logits=True`会在计算损失之前自动计算softmax。

另请注意，我们将 `learning_rate` 传递给 `SGD` 优化器。 学习率是梯度下降算法中需要的一个参数。 我们可以将其保留为默认值，即 0.01，但知道如何指定它很重要，因为不同的学习率会导致非常不同的预测准确度。

最后，请注意我们指定了一个 `batch_size`，正如我们之前看到的，我们在构建 `Dataset` 时使用了它。 这在训练期间很重要，因为它告诉模型我们要一次训练 64 张图像。 你可能想知道为什么是 64？ 为什么不一次训练一张图片？ 或者一次全部 60,000 张图像？ 对每个单独的图像进行完整的训练步骤是低效的，因为我们必须执行所有计算 60,000 次才能考虑到每个输入图像。 如果我们将所有输入图像包含在 $X$ 中，我们将需要大量内存，并且我们将花费大量时间计算每个训练步骤。 所以我们选择一个介于两者之间的大小，称为“小批量”大小。

现在我们已经用训练所需的参数配置了模型，我们可以调用 `fit` 来训练模型。 我们将 epoch 数指定为 5，这意味着我们要在训练神经网络时对完整的 60,000 张训练图像集进行五次迭代。

In [4]:
epochs = 5
print('\nFitting:')
model.fit(train_dataset, epochs=epochs)


Fitting:
Epoch 1/5
938/938 [==============================] - 2s 2ms/step - loss: 0.6220 - accuracy: 0.7797
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4599 - accuracy: 0.8352
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4215 - accuracy: 0.8494
Epoch 4/5
938/938 [==============================] - 2s 2ms/step - loss: 0.3995 - accuracy: 0.8566
Epoch 5/5
938/938 [==============================] - 2s 2ms/step - loss: 0.3863 - accuracy: 0.8601


Training has found values for the parameters $W$ and $b$ such that, when we provide an image as input, we'll get a reasonable prediction as output. Our model is now ready to be tested. Remember that when we loaded the data, we obtained two datasets, one with training data and another with test data. It's time to use the test dataset.

训练已经找到参数 $W$ 和 $b$ 的值，这样当我们提供图像作为输入时，我们将得到合理的预测作为输出。 我们的模型现在可以进行测试了。 请记住，当我们加载数据时，我们获得了两个数据集，一个包含训练数据，另一个包含测试数据。 是时候使用测试数据集了。

In [5]:
print('\nEvaluating:')
(test_loss, test_accuracy) = model.evaluate(test_dataset)
print(f'\nTest accuracy: {test_accuracy * 100:>0.1f}%, test loss: {test_loss:>8f}')


Evaluating:
157/157 [==============================] - 0s 638us/step - loss: 0.4626 - accuracy: 0.8252

Test accuracy: 82.5%, test loss: 0.462619


We've achieved pretty good test accuracy, considering that we used such a simple network and only five epochs of training. We're done with training and can now save the model.

考虑到我们使用了如此简单的网络并且只训练了五个时期，我们已经取得了相当不错的测试准确性。 我们完成了训练，现在可以保存模型了。

In [6]:
  model.save('outputs/model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: outputs/model/assets


Now that our neural network has appropriate values for its parameters, we can use it to make a prediction.

现在我们的神经网络为其参数设置了适当的值，我们可以使用它来进行预测。